In [1]:
import pandas as pd
import numpy as np
# import pywedge as pw
from pathlib import Path
# from data_dashboard import Dashboard

import matplotlib.pyplot as plt
import seaborn as sns

import lightgbm
import sklearn.linear_model as lm
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from ydata_profiling import ProfileReport
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
import plotly.express as px
from tqdm import tqdm

from pylab import savefig
import shap

In [2]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

In [3]:
path_to_data = Path('/home/kosintsev_ag@sib.evraz.com/PycharmProjects/2023_evraz_hackaton/exploration/data/kafka_data.csv')
data = pd.read_csv(path_to_data)
mapping = pd.read_excel('/home/kosintsev_ag@sib.evraz.com/PycharmProjects/2023_evraz_hackaton/exploration/data/Маппинг сигналов.xlsx')
mapping.index = mapping['Код сигнала в Kafka']
mapping.drop('Код сигнала в Kafka', axis=1, inplace=True)

In [4]:
mapping_dict = mapping.to_dict()['Название']
print(str(mapping_dict)[:200])

{'SM_Exgauster\\[2:27]': 'Температура нагрева подшипника №1 (ротора привода эксгаустера №1)', 'SM_Exgauster\\[2:65]': 'Уставка аварийная max ТП т1 Эксгаустер №1', 'SM_Exgauster\\[2:74]': 'Уставка авар


In [5]:
print('Найден мапинг для датчиков:', len([True for i in set(data.columns) if i in set(mapping_dict.keys())]))
missed_map_cols = [i for i in set(data.columns) if i not in set(mapping_dict.keys())]
print('Не найден мапинг для датчиков:', missed_map_cols)

Найден мапинг для датчиков: 718
Не найден мапинг для датчиков: ['SM_Exgauster\\[8:0]', 'SM_Exgauster\\[7:2]', 'SM_Exgauster\\[5.0]', 'SM_Exgauster\\[3:58]', 'SM_Exgauster\\[2:57]', 'SM_Exgauster\\[3:57]', 'SM_Exgauster\\[2:63]', 'SM_Exgauster\\[1.5]', 'SM_Exgauster\\[0:58]', 'SM_Exgauster\\[1.3]', 'SM_Exgauster\\[2:46]', 'SM_Exgauster\\[6:0]', 'SM_Exgauster\\[5.8]', 'SM_Exgauster\\[6.0]', 'SM_Exgauster\\[4.0]', 'SM_Exgauster\\[3:56]', 'SM_Exgauster\\[4.8]', 'SM_Exgauster\\[3:55]', 'SM_Exgauster\\[8.0]', 'SM_Exgauster\\[2:56]', 'SM_Exgauster\\[4.5]', 'SM_Exgauster\\[1.0]', 'SM_Exgauster\\[7:0]', 'SM_Exgauster\\[0:56]', 'SM_Exgauster\\[6:1]', 'SM_Exgauster\\[5.5]', 'SM_Exgauster\\[1.4]', 'SM_Exgauster\\[7.0]', 'SM_Exgauster\\[1.9]', 'SM_Exgauster\\[0:57]', 'SM_Exgauster\\[3:46]', 'SM_Exgauster\\[7:1]', 'SM_Exgauster\\[6:2]', 'SM_Exgauster\\[8:1]', 'SM_Exgauster\\[2:64]', 'SM_Exgauster\\[8:2]', 'SM_Exgauster\\[2:58]', 'SM_Exgauster\\[5.3]', 'SM_Exgauster\\[4.9]', 'SM_Exgauster\\[4.3]', 'S

In [6]:
print('shape:\n', data.shape)
data = data.rename(columns=mapping_dict)
data.index = data['moment']
# data.drop('moment', axis=1, inplace=True)
data.drop(missed_map_cols, axis=1, inplace=True)

print('shape:\n', data.shape)
print('\n', data.info())
data.head(5)

shape:
 (31281, 766)
shape:
 (31281, 718)
<class 'pandas.core.frame.DataFrame'>
Index: 31281 entries, 2023-01-25T06:32:25.125007 to 2023-02-17T04:42:25.498636
Columns: 718 entries, Горизонтал вибрация подшипника №1. Эксгаустер №3 to ОТКРЫТО задвижка газ эксгаустер №6
dtypes: float64(718)
memory usage: 171.6+ MB

 None


,Горизонтал вибрация подшипника №1. Эксгаустер №3,Вертикал вибрация подшипника №1. Эксгаустер №3,Осевая вибрация подшипника №1. Эксгаустер №3,Горизонтал вибрация подшипника №2. Эксгаустер №3,Вертикал вибрация подшипника №2. Эксгаустер №3,Осевая вибрация подшипника №2. Эксгаустер №3,Горизонтал вибрация подшипника №7. Эксгаустер №3,Вертикал вибрация подшипника №7. Эксгаустер №3,Осевая вибрация подшипника №7. Эксгаустер №3,Горизонтал вибрация подшипника №8. Эксгаустер №3,Вертикал вибрация подшипника №8. Эксгаустер №3,Осевая вибрация подшипника №8. Эксгаустер №3,Горизонтал вибрация подшипника №1. Эксгаустер №4,Вертикал вибрация подшипника №1. Эксгаустер №4,Осевая вибрация подшипника №1. Эксгаустер №4,Горизонтал вибрация подшипника №2. Эксгаустер №4,Вертикал вибрация подшипника №2. Эксгаустер №4,Осевая вибрация подшипника №2. Эксгаустер №4,Горизонтал вибрация подшипника №7. Эксгаустер №4,Вертикал вибрация подшипника №7. Эксгаустер №4,Осевая вибрация подшипника №7. Эксгаустер №4,Горизонтал вибрация подшипника №8. Эксгаустер №4,Вертикал вибрация подшипника №8. Эксгаустер №4,Осевая вибрация подшипника №8. Эксгаустер №4,Температура перед эксгаустером №3,Температура перед эксгаустером №4,Температура нагрева подшипника №1 (ротора привода эксгаустера №3),Температура нагрева подшипника №2 (ротора привода эксгаустера №3),Температура нагрева подшипника №3 (редуктор эксгаустера). Эксгаустер №3,Температура нагрева подшипника №4 (редуктор эксгаустера №3),Температура нагрева подшипника №5 (редуктор эксгаустера №3),Температура нагрева подшипника №6 (редуктор эксгаустера №3),Температура нагрева подшипника №7 (ротор эксгаустера №3),Температура нагрева подшипника №8 (ротор эксгаустера №3),Температура нагрева подшипника №9 (ротор эксгаустера №3),Температура воды до охладителя. Эксгаустер №3,Температура воды на выходе охладителя. Эксгаустер №3,Температура масла до охладителя. Эксгаустер №3,Температура масла после охладителя. Эксгаустер №3,Температура нагрева подшипника №1 (ротора привода эксгаустера №4),Температура нагрева подшипника №2 (ротора привода эксгаустера №4),Температура нагрева подшипника №3 (редуктор эксгаустера №4),Температура нагрева подшипника №4 (редуктор эксгаустера №4),Температура нагрева подшипника №5 (редуктор эксгаустера №4),Температура нагрева подшипника №6 (редуктор эксгаустера №4),Температура нагрева подшипника №7 (ротор эксгаустера №4),Температура нагрева подшипника №8 (ротор эксгаустера №4),Температура нагрева подшипника №9 (ротор эксгаустера №4),Температура воды до охладителя. Эксгаустер №4,Температура воды на выходе охладителя. Эксгаустер №4,Температура масла до охладителя. Эксгаустер №4,Температура масла после охладителя. Эксгаустер №4,Разрежение перед эксгаустером №3,Разрежение перед эксгаустером №4,Уставка аварийная max ТП т1 Эксгаустер №3,Уставка аварийная max ТП т2 Эксгаустер №3,Уставка аварийная max ТП т3 Эксгаустер №3,Уставка аварийная max ТП т4 Эксгаустер №3,Уставка аварийная max ТП т5 Эксгаустер №3,Уставка аварийная max ТП т6 Эксгаустер №3,Уставка аварийная max ТП т7 Эксгаустер №3,Уставка аварийная max ТП т8 Эксгаустер №3,Уставка аварийная max ТП т9 Эксгаустер №3,Уставка аварийная min ТП т1 Эксгаустер №3,Уставка аварийная min ТП т2 Эксгаустер №3,Уставка аварийная min ТП т3 Эксгаустер №3,Уставка аварийная min ТП т4 Эксгаустер №3,Уставка аварийная min ТП т5 Эксгаустер №3,Уставка аварийная min ТП т6 Эксгаустер №3,Уставка аварийная min ТП т7 Эксгаустер №3,Уставка аварийная min ТП т8 Эксгаустер №3,Уставка аварийная min ТП т9 Эксгаустер №3,Уставка предупредительная max ТП т1 Эксгаустер №3,Уставка предупредительная max ТП т2 Эксгаустер №3,Уставка предупредительная max ТП т3 Эксгаустер №3,Уставка предупредительная max ТП т4 Эксгаустер №3,Уставка предупредительная max ТП т5 Эксгаустер №3,Уставка предупредительная max ТП т6 Эксгаустер №3,Уставка предупредительная max ТП т7 Эксгаустер №3,Уставка предупредительная max ТП т8 Эксгаустер №3,Уставка предупредительная max ТП т9 Эксгаустер №3,Уставка предупредительная min ТП т1 Эксга

In [7]:
data.describe()

,Горизонтал вибрация подшипника №1. Эксгаустер №3,Вертикал вибрация подшипника №1. Эксгаустер №3,Осевая вибрация подшипника №1. Эксгаустер №3,Горизонтал вибрация подшипника №2. Эксгаустер №3,Вертикал вибрация подшипника №2. Эксгаустер №3,Осевая вибрация подшипника №2. Эксгаустер №3,Горизонтал вибрация подшипника №7. Эксгаустер №3,Вертикал вибрация подшипника №7. Эксгаустер №3,Осевая вибрация подшипника №7. Эксгаустер №3,Горизонтал вибрация подшипника №8. Эксгаустер №3,Вертикал вибрация подшипника №8. Эксгаустер №3,Осевая вибрация подшипника №8. Эксгаустер №3,Горизонтал вибрация подшипника №1. Эксгаустер №4,Вертикал вибрация подшипника №1. Эксгаустер №4,Осевая вибрация подшипника №1. Эксгаустер №4,Горизонтал вибрация подшипника №2. Эксгаустер №4,Вертикал вибрация подшипника №2. Эксгаустер №4,Осевая вибрация подшипника №2. Эксгаустер №4,Горизонтал вибрация подшипника №7. Эксгаустер №4,Вертикал вибрация подшипника №7. Эксгаустер №4,Осевая вибрация подшипника №7. Эксгаустер №4,Горизонтал вибрация подшипника №8. Эксгаустер №4,Вертикал вибрация подшипника №8. Эксгаустер №4,Осевая вибрация подшипника №8. Эксгаустер №4,Температура перед эксгаустером №3,Температура перед эксгаустером №4,Температура нагрева подшипника №1 (ротора привода эксгаустера №3),Температура нагрева подшипника №2 (ротора привода эксгаустера №3),Температура нагрева подшипника №3 (редуктор эксгаустера). Эксгаустер №3,Температура нагрева подшипника №4 (редуктор эксгаустера №3),Температура нагрева подшипника №5 (редуктор эксгаустера №3),Температура нагрева подшипника №6 (редуктор эксгаустера №3),Температура нагрева подшипника №7 (ротор эксгаустера №3),Температура нагрева подшипника №8 (ротор эксгаустера №3),Температура нагрева подшипника №9 (ротор эксгаустера №3),Температура воды до охладителя. Эксгаустер №3,Температура воды на выходе охладителя. Эксгаустер №3,Температура масла до охладителя. Эксгаустер №3,Температура масла после охладителя. Эксгаустер №3,Температура нагрева подшипника №1 (ротора привода эксгаустера №4),Температура нагрева подшипника №2 (ротора привода эксгаустера №4),Температура нагрева подшипника №3 (редуктор эксгаустера №4),Температура нагрева подшипника №4 (редуктор эксгаустера №4),Температура нагрева подшипника №5 (редуктор эксгаустера №4),Температура нагрева подшипника №6 (редуктор эксгаустера №4),Температура нагрева подшипника №7 (ротор эксгаустера №4),Температура нагрева подшипника №8 (ротор эксгаустера №4),Температура нагрева подшипника №9 (ротор эксгаустера №4),Температура воды до охладителя. Эксгаустер №4,Температура воды на выходе охладителя. Эксгаустер №4,Температура масла до охладителя. Эксгаустер №4,Температура масла после охладителя. Эксгаустер №4,Разрежение перед эксгаустером №3,Разрежение перед эксгаустером №4,Уставка аварийная max ТП т1 Эксгаустер №3,Уставка аварийная max ТП т2 Эксгаустер №3,Уставка аварийная max ТП т3 Эксгаустер №3,Уставка аварийная max ТП т4 Эксгаустер №3,Уставка аварийная max ТП т5 Эксгаустер №3,Уставка аварийная max ТП т6 Эксгаустер №3,Уставка аварийная max ТП т7 Эксгаустер №3,Уставка аварийная max ТП т8 Эксгаустер №3,Уставка аварийная max ТП т9 Эксгаустер №3,Уставка аварийная min ТП т1 Эксгаустер №3,Уставка аварийная min ТП т2 Эксгаустер №3,Уставка аварийная min ТП т3 Эксгаустер №3,Уставка аварийная min ТП т4 Эксгаустер №3,Уставка аварийная min ТП т5 Эксгаустер №3,Уставка аварийная min ТП т6 Эксгаустер №3,Уставка аварийная min ТП т7 Эксгаустер №3,Уставка аварийная min ТП т8 Эксгаустер №3,Уставка аварийная min ТП т9 Эксгаустер №3,Уставка предупредительная max ТП т1 Эксгаустер №3,Уставка предупредительная max ТП т2 Эксгаустер №3,Уставка предупредительная max ТП т3 Эксгаустер №3,Уставка предупредительная max ТП т4 Эксгаустер №3,Уставка предупредительная max ТП т5 Эксгаустер №3,Уставка предупредительная max ТП т6 Эксгаустер №3,Уставка предупредительная max ТП т7 Эксгаустер №3,Уставка предупредительная max ТП т8 Эксгаустер №3,Уставка предупредительная max ТП т9 Эксгаустер №3,Уставка предупредительная min ТП т1 Эксга

In [21]:
def plot_scatters(df, plot_cols=None, save=False):
    plot_cols = plot_cols or df.columns
    for col in tqdm(plot_cols):

        scatter = px.scatter(df, x=df.index.values, y=str(col), opacity=0.3)
        scatter.update_layout(xaxis_title="index", yaxis_title=str(col))
        if save:
            scatter.write_image(Path("EDA") / "scatter" / f"{col}.png")

        else:
            scatter.show()

def plot_lines(df, plot_cols=None, save=False):
    plot_cols = plot_cols or df.columns
    for col in tqdm(plot_cols):

        line = px.line(df, x=df.index.values, y=str(col), markers=True)
        line.update_layout(xaxis_title="index", yaxis_title=str(col))
        if save:
            line.write_image(Path("EDA") / "line" / f"{col}.png")

        else:
            line.show()

def plot_hists(df, plot_cols):
    for col in plot_cols:
        f, ax = plt.subplots(figsize=(10,6))
        ax.set_title(col)
        df[col].hist(bins=100)

def isna_percent(df):
    return (df.isna().sum()/df.shape[0]*100).sort_values(ascending=False)

def isna_count(df):
    return df.isna().sum().sort_values(ascending=False)

In [9]:
isna_count(data)

Горизонтал вибрация подшипника №1. Эксгаустер №3     2
Уставка аварийная min ОВ 7т Эксгаустер №4            2
Уставка предупредительная min ОВ 1т Эксгаустер №3    2
Уставка предупредительная min ГВ 2т Эксгаустер №3    2
Уставка предупредительная min ВВ 2т Эксгаустер №3    2
                                                    ..
Уставка аварийная max ОВ 8т Эксгаустер №2            0
Уставка аварийная max ВВ 8т Эксгаустер №2            0
Уставка аварийная max ГВ 8т Эксгаустер №2            0
Уставка аварийная max ОВ 7т Эксгаустер №2            0
ОТКРЫТО задвижка газ эксгаустер №6                   0
Length: 718, dtype: int64

In [10]:
data = data.dropna().sort_index()
data.head()

,Горизонтал вибрация подшипника №1. Эксгаустер №3,Вертикал вибрация подшипника №1. Эксгаустер №3,Осевая вибрация подшипника №1. Эксгаустер №3,Горизонтал вибрация подшипника №2. Эксгаустер №3,Вертикал вибрация подшипника №2. Эксгаустер №3,Осевая вибрация подшипника №2. Эксгаустер №3,Горизонтал вибрация подшипника №7. Эксгаустер №3,Вертикал вибрация подшипника №7. Эксгаустер №3,Осевая вибрация подшипника №7. Эксгаустер №3,Горизонтал вибрация подшипника №8. Эксгаустер №3,Вертикал вибрация подшипника №8. Эксгаустер №3,Осевая вибрация подшипника №8. Эксгаустер №3,Горизонтал вибрация подшипника №1. Эксгаустер №4,Вертикал вибрация подшипника №1. Эксгаустер №4,Осевая вибрация подшипника №1. Эксгаустер №4,Горизонтал вибрация подшипника №2. Эксгаустер №4,Вертикал вибрация подшипника №2. Эксгаустер №4,Осевая вибрация подшипника №2. Эксгаустер №4,Горизонтал вибрация подшипника №7. Эксгаустер №4,Вертикал вибрация подшипника №7. Эксгаустер №4,Осевая вибрация подшипника №7. Эксгаустер №4,Горизонтал вибрация подшипника №8. Эксгаустер №4,Вертикал вибрация подшипника №8. Эксгаустер №4,Осевая вибрация подшипника №8. Эксгаустер №4,Температура перед эксгаустером №3,Температура перед эксгаустером №4,Температура нагрева подшипника №1 (ротора привода эксгаустера №3),Температура нагрева подшипника №2 (ротора привода эксгаустера №3),Температура нагрева подшипника №3 (редуктор эксгаустера). Эксгаустер №3,Температура нагрева подшипника №4 (редуктор эксгаустера №3),Температура нагрева подшипника №5 (редуктор эксгаустера №3),Температура нагрева подшипника №6 (редуктор эксгаустера №3),Температура нагрева подшипника №7 (ротор эксгаустера №3),Температура нагрева подшипника №8 (ротор эксгаустера №3),Температура нагрева подшипника №9 (ротор эксгаустера №3),Температура воды до охладителя. Эксгаустер №3,Температура воды на выходе охладителя. Эксгаустер №3,Температура масла до охладителя. Эксгаустер №3,Температура масла после охладителя. Эксгаустер №3,Температура нагрева подшипника №1 (ротора привода эксгаустера №4),Температура нагрева подшипника №2 (ротора привода эксгаустера №4),Температура нагрева подшипника №3 (редуктор эксгаустера №4),Температура нагрева подшипника №4 (редуктор эксгаустера №4),Температура нагрева подшипника №5 (редуктор эксгаустера №4),Температура нагрева подшипника №6 (редуктор эксгаустера №4),Температура нагрева подшипника №7 (ротор эксгаустера №4),Температура нагрева подшипника №8 (ротор эксгаустера №4),Температура нагрева подшипника №9 (ротор эксгаустера №4),Температура воды до охладителя. Эксгаустер №4,Температура воды на выходе охладителя. Эксгаустер №4,Температура масла до охладителя. Эксгаустер №4,Температура масла после охладителя. Эксгаустер №4,Разрежение перед эксгаустером №3,Разрежение перед эксгаустером №4,Уставка аварийная max ТП т1 Эксгаустер №3,Уставка аварийная max ТП т2 Эксгаустер №3,Уставка аварийная max ТП т3 Эксгаустер №3,Уставка аварийная max ТП т4 Эксгаустер №3,Уставка аварийная max ТП т5 Эксгаустер №3,Уставка аварийная max ТП т6 Эксгаустер №3,Уставка аварийная max ТП т7 Эксгаустер №3,Уставка аварийная max ТП т8 Эксгаустер №3,Уставка аварийная max ТП т9 Эксгаустер №3,Уставка аварийная min ТП т1 Эксгаустер №3,Уставка аварийная min ТП т2 Эксгаустер №3,Уставка аварийная min ТП т3 Эксгаустер №3,Уставка аварийная min ТП т4 Эксгаустер №3,Уставка аварийная min ТП т5 Эксгаустер №3,Уставка аварийная min ТП т6 Эксгаустер №3,Уставка аварийная min ТП т7 Эксгаустер №3,Уставка аварийная min ТП т8 Эксгаустер №3,Уставка аварийная min ТП т9 Эксгаустер №3,Уставка предупредительная max ТП т1 Эксгаустер №3,Уставка предупредительная max ТП т2 Эксгаустер №3,Уставка предупредительная max ТП т3 Эксгаустер №3,Уставка предупредительная max ТП т4 Эксгаустер №3,Уставка предупредительная max ТП т5 Эксгаустер №3,Уставка предупредительная max ТП т6 Эксгаустер №3,Уставка предупредительная max ТП т7 Эксгаустер №3,Уставка предупредительная max ТП т8 Эксгаустер №3,Уставка предупредительная max ТП т9 Эксгаустер №3,Уставка предупредительная min ТП т1 Эксга

In [11]:
all_ustavki = [i for i in data.columns if 'уставк' in i.lower()]
us_ravni = (data[all_ustavki] == data[all_ustavki].iloc[0]).min()
us_ravni[us_ravni==False]

Уставка аварийная max ГВ 7т Эксгаустер №4    False
dtype: bool

In [12]:
a = data['Уставка аварийная max ГВ 7т Эксгаустер №4'] - data['Уставка аварийная max ГВ 7т Эксгаустер №4'].shift(-1)
a[a!=0]

moment
2023-02-06T07:05:25.599340   -5.9
2023-02-11T23:23:25.116255    5.9
2023-02-17T04:42:25.498636    NaN
Name: Уставка аварийная max ГВ 7т Эксгаустер №4, dtype: float64

##### Всего 1 уставка отличается в истории, менялась туда обратно 1 раз. Все уставки сбросим с data

In [13]:
ustavki = data[all_ustavki].copy()
data.drop(all_ustavki, axis=1, inplace=True)
data.index = data.index.astype('datetime64[ns]')
data.columns = data.columns.str.lower()

In [14]:
data.head()

,горизонтал вибрация подшипника №1. эксгаустер №3,вертикал вибрация подшипника №1. эксгаустер №3,осевая вибрация подшипника №1. эксгаустер №3,горизонтал вибрация подшипника №2. эксгаустер №3,вертикал вибрация подшипника №2. эксгаустер №3,осевая вибрация подшипника №2. эксгаустер №3,горизонтал вибрация подшипника №7. эксгаустер №3,вертикал вибрация подшипника №7. эксгаустер №3,осевая вибрация подшипника №7. эксгаустер №3,горизонтал вибрация подшипника №8. эксгаустер №3,вертикал вибрация подшипника №8. эксгаустер №3,осевая вибрация подшипника №8. эксгаустер №3,горизонтал вибрация подшипника №1. эксгаустер №4,вертикал вибрация подшипника №1. эксгаустер №4,осевая вибрация подшипника №1. эксгаустер №4,горизонтал вибрация подшипника №2. эксгаустер №4,вертикал вибрация подшипника №2. эксгаустер №4,осевая вибрация подшипника №2. эксгаустер №4,горизонтал вибрация подшипника №7. эксгаустер №4,вертикал вибрация подшипника №7. эксгаустер №4,осевая вибрация подшипника №7. эксгаустер №4,горизонтал вибрация подшипника №8. эксгаустер №4,вертикал вибрация подшипника №8. эксгаустер №4,осевая вибрация подшипника №8. эксгаустер №4,температура перед эксгаустером №3,температура перед эксгаустером №4,температура нагрева подшипника №1 (ротора привода эксгаустера №3),температура нагрева подшипника №2 (ротора привода эксгаустера №3),температура нагрева подшипника №3 (редуктор эксгаустера). эксгаустер №3,температура нагрева подшипника №4 (редуктор эксгаустера №3),температура нагрева подшипника №5 (редуктор эксгаустера №3),температура нагрева подшипника №6 (редуктор эксгаустера №3),температура нагрева подшипника №7 (ротор эксгаустера №3),температура нагрева подшипника №8 (ротор эксгаустера №3),температура нагрева подшипника №9 (ротор эксгаустера №3),температура воды до охладителя. эксгаустер №3,температура воды на выходе охладителя. эксгаустер №3,температура масла до охладителя. эксгаустер №3,температура масла после охладителя. эксгаустер №3,температура нагрева подшипника №1 (ротора привода эксгаустера №4),температура нагрева подшипника №2 (ротора привода эксгаустера №4),температура нагрева подшипника №3 (редуктор эксгаустера №4),температура нагрева подшипника №4 (редуктор эксгаустера №4),температура нагрева подшипника №5 (редуктор эксгаустера №4),температура нагрева подшипника №6 (редуктор эксгаустера №4),температура нагрева подшипника №7 (ротор эксгаустера №4),температура нагрева подшипника №8 (ротор эксгаустера №4),температура нагрева подшипника №9 (ротор эксгаустера №4),температура воды до охладителя. эксгаустер №4,температура воды на выходе охладителя. эксгаустер №4,температура масла до охладителя. эксгаустер №4,температура масла после охладителя. эксгаустер №4,разрежение перед эксгаустером №3,разрежение перед эксгаустером №4,46 работа эксгаустера №3,62 работа эксгаустера №4,уровень масла в маслосистеме эксгаустера №3,давление масла в маслосистеме эксгаустера №3,ток ротора эксгаустера №3,ток статора эксгаустера №3,напряжение ротора эксгаустера №3,напряжение статора эксгаустера №3,положение задвижки газ эксгаустера №3,уровень масла в маслосистеме эксгаустера №4,давление масла в маслосистеме эксгаустера №4,ток ротора эксгаустера №4,ток статора эксгаустера №4,напряжение ротора эксгаустера №4,напряжение статора эксгаустера №4,положение задвижки газ эксгаустера №4,закрыто задвижка газ эксгаустер №3,открыто задвижка газ эксгаустер №3,закрыто задвижка газ эксгаустер №4,открыто задвижка газ эксгаустер №4,горизонтал вибрация подшипника №1. эксгаустер №1,вертикал вибрация подшипника №1. эксгаустер №1,осевая вибрация подшипника №1. эксгаустер №1,горизонтал вибрация подшипника №2. эксгаустер №1,вертикал вибрация подшипника №2. эксгаустер №1,осевая вибрация подшипника №2. эксгаустер №1,горизонтал вибрация подшипника №7. эксгаустер №1,вертикал вибрация подшипника №7. эксгаустер №1,осевая вибрация подшипника №7. эксгаустер №1,горизонтал вибрация подшипника №8. эксгаустер №1,вертикал вибрация подшипника №8. эксгаустер №1,осевая вибрация подшипника №8. эксгаустер №1

In [15]:
data.shape

(31279, 214)

### Базовый анализ по данным температур, вибраций, и т.д.

In [23]:
plot_lines(data, save=True)

100%|██████████| 214/214 [04:55<00:00,  1.38s/it]


### Почистим выбросы:
Для вибраций есть небольшое смещение ноля на датчике

In [35]:
corr_range = {
    'вертикал вибрация подшипника №7. эксгаустер №3': {'min': -0.1, 'max': np.inf},
    'вертикал вибрация подшипника №7. эксгаустер №4': {'min': -0.1, 'max': np.inf},
    'вертикал вибрация подшипника №7. эксгаустер №5': {'min': -0.1, 'max': np.inf},
    'вертикал вибрация подшипника №8. эксгаустер №3': {'min': -0.1, 'max': np.inf},
    'вертикал вибрация подшипника №8. эксгаустер №4': {'min': -0.1, 'max': np.inf},
    'горизонтал вибрация подшипника №7. эксгаустер №4': {'min': -0.1, 'max': np.inf},
    'горизонтал вибрация подшипника №8. эксгаустер №1': {'min': -0.1, 'max': np.inf},
    'напряжение ротора эксгаустера №5': {'min': -100, 'max': np.inf},
    'напряжение ротора эксгаустера №6': {'min': -100, 'max': np.inf},
    'температура воды до охладителя. эксгаустер №1': {'min': 0, 'max': 50},
    'температура воды до охладителя. эксгаустер №2': {'min': 0, 'max': 50},
    'температура масла до охладителя. эксгаустер №1': {'min': -100, 'max': 100},
    'температура масла до охладителя. эксгаустер №2': {'min': -100, 'max': 100},
    'температура масла до охладителя. эксгаустер №5': {'min': -100, 'max': 100},
    'температура масла после охладителя. эксгаустер №1': {'min': -100, 'max': 100},
    'температура масла после охладителя. эксгаустер №2': {'min': -100, 'max': 100},
    'температура масла после охладителя. эксгаустер №5': {'min': -100, 'max': 100},
    'уровень масла в маслосистеме эксгаустера №6': {'min': 0, 'max': np.inf},
}

In [38]:
data_filtered = data.copy()

for c in corr_range.keys():
    row_before = data_filtered.shape[0]
    data_filtered = data_filtered[
        (data_filtered[c] >= corr_range[c]['min'])
        & (data_filtered[c] <= corr_range[c]['max'])
    ]
    row_after = data_filtered.shape[0]
    print(f'По столбцу {c} сброшено строк: {row_before-row_after}')

По столбцу вертикал вибрация подшипника №7. эксгаустер №3 сброшено строк: 0
По столбцу вертикал вибрация подшипника №7. эксгаустер №4 сброшено строк: 24
По столбцу вертикал вибрация подшипника №7. эксгаустер №5 сброшено строк: 0
По столбцу вертикал вибрация подшипника №8. эксгаустер №3 сброшено строк: 0
По столбцу вертикал вибрация подшипника №8. эксгаустер №4 сброшено строк: 0
По столбцу горизонтал вибрация подшипника №7. эксгаустер №4 сброшено строк: 11
По столбцу горизонтал вибрация подшипника №8. эксгаустер №1 сброшено строк: 23
По столбцу напряжение ротора эксгаустера №5 сброшено строк: 1
По столбцу напряжение ротора эксгаустера №6 сброшено строк: 1
По столбцу температура воды до охладителя. эксгаустер №1 сброшено строк: 2
По столбцу температура воды до охладителя. эксгаустер №2 сброшено строк: 4
По столбцу температура масла до охладителя. эксгаустер №1 сброшено строк: 3
По столбцу температура масла до охладителя. эксгаустер №2 сброшено строк: 4
По столбцу температура масла до охл

In [39]:
data_exh = dict()
for exh_id in range(1, 7):
    print(exh_id)
    exhauster_i_cols = [
        i for i in data_filtered.columns if f'ксгаустер №{exh_id}' in i
                                   or f'ксгаустера №{exh_id}' in i
                                   or f'ксгаустером №{exh_id}' in i
    ]
    print(f'столбцов для эксгаустера #{exh_id}', len(exhauster_i_cols))

    data_exh[exh_id] = dict()
    data_exh[exh_id]['data'] = data_filtered[exhauster_i_cols].copy()
    print('размер df:', data_exh[exh_id]['data'].shape)

    vibrations_i = [i for i in data_exh[exh_id]['data'].columns if 'вибрация' in i]
    data_exh[exh_id]['vibrations'] = vibrations_i

    temperatures_i = [i for i in data_exh[exh_id]['data'].columns if 'температур' in i]
    data_exh[exh_id]['temperatures'] = temperatures_i

    electr_i = [i for i in data_exh[exh_id]['data'].columns if 'роток' in i or 'статор' in i]
    data_exh[exh_id]['electr'] = electr_i

1
столбцов для эксгаустера #1 33
размер df: (31167, 33)
2
столбцов для эксгаустера #2 33
размер df: (31167, 33)
3
столбцов для эксгаустера #3 37
размер df: (31167, 37)
4
столбцов для эксгаустера #4 37
размер df: (31167, 37)
5
столбцов для эксгаустера #5 37
размер df: (31167, 37)
6
столбцов для эксгаустера #6 37
размер df: (31167, 37)


In [17]:
# mc = pw.Pywedge_Charts(data_3exh, c=None, y=targets_3[-2])
# charts = mc.make_charts()

In [18]:
# output_directory = 'your_path/dashboard_output'
# X, y, descriptions = data_3exh.drop(targets_3[-2], axis=1), data[targets_3[-2]], None
# # descriptions is an optional argument, more in documentation
# dsh = Dashboard(X, y, output_directory, descriptions)
# dsh.create_dashboard()

#### Pandas profiling

In [19]:
# profile = ProfileReport(data_3exh, title=f"{exhauster_number} exhauster")
# profile.to_file("3_exhauster.html")

#### Проанализируем поведение вибраций по подшипникам связных эксгаустеров

In [ ]:
data_plot = data_exh[1]['data']
vibrations = data_exh[1]['vibrations']
px.line(data_plot[vibrations])

# Обучение моделей

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(data.drop('M25', axis=1), data['M25'], test_size=0.2, shuffle=False)

model = Fedot(problem='regression', timeout=15, preset='best_quality', n_jobs=-1)
model.fit(features=x_train, target=y_train)
prediction = model.predict(features=x_test)
metrics = model.get_metrics(target=y_test)

In [ ]:
metrics

In [ ]:
r2_score(y_test, prediction)

In [ ]:
cols_for_model = []

gbm = lightgbm.LGBMRegressor()
model_list = [gbm, ]


def train_model(model_list, x, y):
    for model in model_list:

        df_r2 = pd.Series()

        x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3)
        model.fit(x_train, y_train)

        y_pred = model.predict(x_test)

        fig, ax = plt.subplots(figsize=(12,8))
        ax.set_title(f"Модель {model}")
        feature_importances = pd.Series(model.feature_importances_, index=x.columns)
        top_vals = feature_importances.nlargest(10).sort_values(ascending=True)
        y_label = [str(i) for i in top_vals.index.values]
        # print(y_label)
        ax.barh(y_label, top_vals.values)
        plt.show()

        mse, mae = np.round(mean_absolute_error(y_test, y_pred),10), np.round(mean_squared_error(y_test, y_pred), 10)
        mse = np.round(mse, 8)
        mae = np.round(mae, 8)

        print("MSE =", mse)
        print("MAE =", mae)
        plt.style.use("ggplot")
        fig, ax = plt.subplots(figsize=(15,15))

        test_r2 = np.round(r2_score(y_test, y_pred)*100, 2)
        labell = f"R2 = {test_r2}%, MAE={mae}"
        ax.set_title(labell)
        ax.set_xlabel("FACT")
        ax.set_ylabel("PREDICTION")
        ax.scatter(y_test, y_pred, c=[[0.5, 0, 0.5, 0.15]],)

        line_45_x = [i for i in [y_test.min(), y_test.max()]]
        line_45_y = [i for i in [y_test.min(), y_test.max()]]
        ax.plot(line_45_x, line_45_y, color="gray")
        df_r2[str(model)] = test_r2
        plt.show()
        print("R2 =", test_r2)